# 特徴ベクトル抽出

## ■定数を設定

In [1]:
CONFIG_PATH = '../config.ini'
MAX_SEQ_LENGTH = 128
PRETRAINED_MODEL_PATH = '../model/bert-wiki-ja/model.ckpt-1400000'
TOKENIZER_MODEL_PATH = '../model/bert-wiki-ja/wiki-ja'
INPUT_TEXTFILE_PATH = './data/input.txt'
OUTPUT_TEXTFILE_PATH = './data/output.txt'
OUTPUT_JSONFILE_PATH = './data/output.json'

## ■モジュールを読み込み

In [2]:
import configparser
import glob
import os
import pandas as pd
import subprocess
import sys
import tarfile 
from urllib.request import urlretrieve
import json
import tempfile
import tensorflow as tf

/home/manba/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/manba/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/manba/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/manba/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: Futu

In [3]:
import numpy as np

In [4]:
sys.path.append("../src")
from utils import str_to_value

In [5]:
sys.path.append("../bert")
import modeling

## ■設定ファイルを読み込み

In [6]:
config = configparser.ConfigParser()
config.read(CONFIG_PATH)

['../config.ini']

In [7]:
FILEURL = config['FINETUNING-DATA']['FILEURL']
FILEPATH = config['FINETUNING-DATA']['FILEPATH']
EXTRACTDIR = config['FINETUNING-DATA']['TEXTDIR']

In [8]:
bert_config_file = tempfile.NamedTemporaryFile(mode='w+t', encoding='utf-8', suffix='.json')
bert_config_file.write(json.dumps({k:str_to_value(v) for k,v in config['BERT-CONFIG'].items()}))
bert_config_file.seek(0)
bert_config_file_path = str(bert_config_file.name)
bert_config = modeling.BertConfig.from_json_file(bert_config_file.name)

## ■入力文書を設定

In [9]:
!echo 'すべての人間は、生れながらにして自由であり、かつ、尊厳と権利とについて平等である。 ||| 人間は、理性と良心とを授けられており、互いに同胞の精神をもって行動しなければならない。' > {INPUT_TEXTFILE_PATH}
!echo '「富士〜」で有名な動物園は？ ||| 「富士サファリパーク」です。' >> {INPUT_TEXTFILE_PATH}

## ■特徴ベクトルを抽出

In [10]:
!python ../src/extract_features.py \
  --input_file={INPUT_TEXTFILE_PATH} \
  --output_file={OUTPUT_TEXTFILE_PATH} \
  --vocab_file={TOKENIZER_MODEL_PATH}.vocab \
  --model_file={TOKENIZER_MODEL_PATH}.model \
  --bert_config_file={bert_config_file.name} \
  --init_checkpoint={PRETRAINED_MODEL_PATH} \
  --layers=-1,-2,-3,-4 \
  --max_seq_length={MAX_SEQ_LENGTH} \
  --batch_size=8

W0813 01:04:49.431474 140567494973248 deprecation_wrapper.py:119] From ../src/extract_features.py:439: The name tf.app.run is deprecated. Please use tf.compat.v1.app.run instead.

Loaded a trained SentencePiece model.
W0813 01:04:49.715533 140567494973248 deprecation_wrapper.py:119] From /home/manba/work/bert-japanese/src/tokenization_sentencepiece.py:115: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W0813 01:04:53.977297 140567494973248 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0813 01:04:53.982577 140567494973248 deprecation_wrapper.py:119] From ../src/extract_features.py:292: The name tf.logging.info is deprecated

## ■結果を確認

### 結果ファイルを読み込み

In [11]:
import json
import codecs

outputs = []
with codecs.open(OUTPUT_TEXTFILE_PATH, 'r', 'utf-8') as fin:
  for line in fin:
    data = json.loads(line)
    outputs.append(data)

with codecs.open(OUTPUT_JSONFILE_PATH,'w', 'utf-8') as fout:
  json.dump(outputs, fout, sort_keys=True, ensure_ascii=False)

### 入力文書の形態素解析結果を確認

In [12]:
for doc in outputs:
  print('文書', (doc['linex_index'] + 1), '/', len(outputs))
  print(' ', ' '.join([ feature['token'].replace('[SEP]', '\n  [SEP]') for feature in doc['features'] ]), '\n')

文書 1 / 2
  [CLS] ▁ すべての 人間 は 、 生 れ ながら にして 自由 であり 、 かつ 、 尊 厳 と 権利 と について 平等 である 。 
  [SEP] ▁ 人間 は 、 理性 と 良 心 と を 授け られており 、 互いに 同胞 の精神 をもって 行動 しなければならない 。 
  [SEP] 

文書 2 / 2
  [CLS] ▁「 富士 〜」 で 有名な 動物園 は ? 
  [SEP] ▁「 富士 サ ファ リ パーク 」 です 。 
  [SEP] 



### Let's have a look at the features of the last layer for the word "人間".

The 0-th token is always [CLS], and the 1st token of a sentence is [▁]. So the word comes in 3rd position.

In [13]:
print(outputs[0]['features'][3]['token'])

人間


The last layer is layer 0, the one before is layer -1, etc...
The embeddings are stored in the *values* entry.

### 「人間」の特徴ベクトルを確認

In [14]:
embeddings = outputs[0]['features'][3]['layers'][0]['values']

print('特徴ベクトルの次元数:', len(embeddings), '\n')
print('ノルム:', np.linalg.norm(np.array(embeddings)), '\n')
print('特徴ベクトル:', embeddings, '\n')

特徴ベクトルの次元数: 768 

ノルム: 18.256403441334193 

特徴ベクトル: [0.388213, -0.576151, -0.027084, -1.431741, 0.688938, -0.304473, 0.373505, -0.109832, -1.621072, -0.848877, -0.012777, 0.623844, -0.849169, -0.370514, -0.448757, 0.227168, -0.310765, 0.164168, -0.416387, -0.782097, 0.088901, 0.643659, 0.621601, 0.127453, 0.550596, -0.554063, 0.70033, -0.938487, 0.226078, -0.208362, 0.530374, -0.31495, 0.083271, -0.874866, 0.581575, -0.95967, 0.27626, 0.276187, 0.165083, -0.124819, -0.733634, 0.227352, 0.148887, 0.574101, -1.069794, -0.287446, 0.100649, -0.134502, 0.713487, -0.330703, 0.222692, -0.49073, -0.672788, 0.210758, -1.07941, -0.09508, 0.144539, 1.00762, 0.134064, 0.637895, -0.015204, -0.734581, 0.100435, -0.556787, -0.090496, -0.129595, -0.217037, 0.679371, 1.321034, 0.359275, 0.427882, -0.943426, -0.582211, -0.607388, -0.0033, 0.962177, 0.095276, -0.105843, 0.328633, 0.056413, -1.07188, 0.413312, 0.228835, -0.378046, 0.222511, -0.37087, 0.328615, 0.486272, 0.304307, 0.758792, -0.224085, 0.64